<a href="https://colab.research.google.com/github/VinitVpANDEY/Neural-Network/blob/main/MakeMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
words = open('names.txt', 'r').read().splitlines()
print(words[:5])
print(len(words))
print(min(len(w) for w in words))
print(max(len(w) for w in words))

['emma', 'olivia', 'ava', 'isabella', 'sophia']
32033
2
15


In [ ]:
for w in words[:3]:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1,ch2 in zip(chs, chs[1:]):
    print(ch1, ch2)

<S> e
e m
m m
m a
a <E>
<S> o
o l
l i
i v
v i
i a
a <E>
<S> a
a v
v a
a <E>


In [ ]:
b = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1,ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

sorted_b = sorted(b.items(), key=lambda x: x[1], reverse=True)

for k, v in sorted_b[:5]:
    print(k, v)

('n', '<E>') 6763
('a', '<E>') 6640
('a', 'n') 5438
('<S>', 'a') 4410
('e', '<E>') 3983


In [ ]:
import torch

N = torch.zeros((27,27), dtype=torch.int32)
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0        # we are going to replace special token <S> and <E> with .
itos = {i:s for s,i in stoi.items()}

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1][ix2] += 1

In [ ]:
P = (N+1).float()
P = P / P.sum(1, keepdim=True)

g = torch.Generator().manual_seed(2147483647)
for i in range(10):
  out = []
  ix = 0
  while True:
    p = P[ix]    # accessing the probability density of ix'th row
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break

  print(''.join(out))


junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.


In [ ]:
log_likelihood = 0.0
n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1][ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-559951.5625)
nll=tensor(559951.5625)
2.4543561935424805


In [ ]:
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

print(xs[:5])
print(ys[:5])

tensor([ 0,  5, 13, 13,  1])
tensor([ 5, 13, 13,  1,  0])


In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc.shape

torch.Size([228146, 27])

In [ ]:
# 27 neuron each taking 27-D input
W = torch.randn((27, 27))
logits = xenc @ W   # 228146, 27 => Each row shows up amount of firing for all 27 character for a given input character
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)



In [ ]:
def train(xenc, ys, epochs, lr, W=None):
  if W is None:
    W = torch.randn((27, 27), requires_grad=True)
  else:
    W.requires_grad_(True)

  for epoch in range(epochs):
    # Set learning rate based on epoch
    lr = 1 if epoch < 600 else 0.5

    # forward pass
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()

    if epoch % 10 == 0:
      print(f"Epoch: {epoch}, Loss: {loss.item():.4f}, LR: {lr}")

    # backward pass
    W.grad = None
    loss.backward()

    # update
    W.data += -lr * W.grad

  return W


model = train(xenc, ys, epochs=1000, lr=0.1)

Epoch: 0, Loss: 3.8686, LR: 1
Epoch: 10, Loss: 3.7535, LR: 1
Epoch: 20, Loss: 3.6530, LR: 1
Epoch: 30, Loss: 3.5644, LR: 1
Epoch: 40, Loss: 3.4860, LR: 1
Epoch: 50, Loss: 3.4164, LR: 1
Epoch: 60, Loss: 3.3545, LR: 1
Epoch: 70, Loss: 3.2993, LR: 1
Epoch: 80, Loss: 3.2499, LR: 1
Epoch: 90, Loss: 3.2055, LR: 1
Epoch: 100, Loss: 3.1654, LR: 1
Epoch: 110, Loss: 3.1291, LR: 1
Epoch: 120, Loss: 3.0959, LR: 1
Epoch: 130, Loss: 3.0655, LR: 1
Epoch: 140, Loss: 3.0376, LR: 1
Epoch: 150, Loss: 3.0118, LR: 1
Epoch: 160, Loss: 2.9879, LR: 1
Epoch: 170, Loss: 2.9658, LR: 1
Epoch: 180, Loss: 2.9452, LR: 1
Epoch: 190, Loss: 2.9260, LR: 1
Epoch: 200, Loss: 2.9081, LR: 1
Epoch: 210, Loss: 2.8913, LR: 1
Epoch: 220, Loss: 2.8756, LR: 1
Epoch: 230, Loss: 2.8609, LR: 1
Epoch: 240, Loss: 2.8471, LR: 1
Epoch: 250, Loss: 2.8341, LR: 1
Epoch: 260, Loss: 2.8219, LR: 1
Epoch: 270, Loss: 2.8104, LR: 1
Epoch: 280, Loss: 2.7996, LR: 1
Epoch: 290, Loss: 2.7893, LR: 1
Epoch: 300, Loss: 2.7797, LR: 1
Epoch: 310, Loss: 2

In [ ]:
def generate_name(count):
  for i in range(count):
    out = []
    ix = 0
    while True:
      xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
      logits = xenc @ model
      counts = logits.exp()
      p = counts/ counts.sum(1, keepdims=True)
      ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix])
      if ix == 0:
        break
    print(''.join(out))

generate_name(10)

jaue.
a.
kwpejbionzqppwarbihisisajbrakamorvxqkadeela.
az.
arileri.
chaiadayra.
fbrlqdo.
meyjon.
zqzrabran.
han.
